In [337]:
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *
from lasagne.objectives import *
from lasagne.random import get_rng
from lasagne.updates import *
import numpy as np
import sys
sys.path.append("../../modules/")
import helper as hp

from theano.sandbox.rng_mrg import MRG_RandomStreams as RandomStreams

In [242]:
from theano.ifelse import ifelse

In [192]:
"""
not working yet
"""
class SkipLayer(Layer):
    def __init__(self, incoming, wrapped_layer, p=0.5, **kwargs):
        super(SkipLayer, self).__init__(incoming, **kwargs)
        self._srng = RandomStreams(get_rng().randint(1, 2147462579))
        self.wrapped_layer = wrapped_layer
        self.incoming = incoming
        self.p = p
    def get_output_for(self, input, deterministic=False, **kwargs):
        # if deterministic, use the underlying layer
        if deterministic or self.p == 0:
            return self.wrapped_layer.get_output_for(input, deterministic=False, **kwargs)
        else:
            # if uniform(0,1) < p, skip the underlying layer
            u = np.random.normal(0, 1)
            if u < self.p:
                print "ignoring"
                return input
            else:
                return self.wrapped_layer.get_output_for(input, deterministic=False, **kwargs)

----

In [281]:
class SkippableNonlinearityLayer(Layer):
    def __init__(self, incoming, nonlinearity=rectify, p=0.5,
                 **kwargs):
        super(SkippableNonlinearityLayer, self).__init__(incoming, **kwargs)
        self.nonlinearity = (identity if nonlinearity is None
                             else nonlinearity)
        self._srng = RandomStreams(get_rng().randint(1, 2147462579))
        self.p = p

    def get_output_for(self, input, deterministic=False, **kwargs):
        if deterministic or self.p == 0:
            return self.nonlinearity(input)
        else:
            return ifelse(
                T.lt(self._srng.uniform( (1,), 0, 1)[0], self.p),
                input,
                self.nonlinearity(input)
            )

In [346]:
def get_net(args={}):
    l_in = InputLayer( (None, 1, 28, 28) )
    #l_dense = DenseLayer(l_in, num_units=50)
    #l_skip = SkipLayer(l_in, l_dense)
    l_conv1 = SkippableNonlinearityLayer(
        Conv2DLayer(l_in, filter_size=(3,3), num_filters=16, nonlinearity=linear),
        nonlinearity=rectify,
        p=0.5
    )
    l_mp1 = MaxPool2DLayer(l_conv1, pool_size=(2,2))
    l_conv2 = SkippableNonlinearityLayer(
        Conv2DLayer(l_mp1, filter_size=(3,3), num_filters=32, nonlinearity=linear),
        nonlinearity=rectify,
        p=0.5
    )
    l_mp2 = MaxPool2DLayer(l_conv2, pool_size=(2,2))
    l_conv3 = SkippableNonlinearityLayer(
        Conv2DLayer(l_mp2, filter_size=(3,3), num_filters=64, nonlinearity=linear),
        nonlinearity=rectify,
        p=0.5
    )
    l_out = DenseLayer(l_conv3, num_units=10, nonlinearity=softmax)
    for layer in get_all_layers(l_out):
        print layer.output_shape
    print count_params(l_out)
    # ----
    X = T.tensor4('X')
    y = T.ivector('y')
    net_out = get_output(l_out, X)
    net_out_det = get_output(l_out, X, deterministic=True)
    loss = categorical_crossentropy(net_out, y).mean()
    params = get_all_params(l_out, trainable=True)
    updates = nesterov_momentum(loss, params, learning_rate=0.01, momentum=0.9)
    train_fn = theano.function([X, y], loss, updates=updates)
    eval_fn = theano.function([X], net_out_det)
    return {
        "train_fn": train_fn,
        "eval_fn": eval_fn
    }

In [355]:
net_cfg = get_net()

(None, 1, 28, 28)
(None, 16, 26, 26)
(None, 16, 26, 26)
(None, 16, 13, 13)
(None, 32, 11, 11)
(None, 32, 11, 11)
(None, 32, 5, 5)
(None, 64, 3, 3)
(None, 64, 3, 3)
(None, 10)
29066


In [344]:
train_data, _, _ = hp.load_mnist("../../data/mnist.pkl.gz")
X_train, y_train = train_data
X_train = X_train.astype("float32")
y_train = y_train.astype("int32")

In [356]:
bs=128
for epoch in range(0, 10):
    b = 0
    train_losses = []
    while True:
        if b*bs >= X_train.shape[0]:
            break
        train_losses.append(
            net_cfg["train_fn"](X_train[b*bs:(b+1)*bs], y_train[b*bs:(b+1)*bs])
        )
        b += 1
    print np.mean(train_losses)

0.454405799224
0.117364909076


KeyboardInterrupt: 